# Scanpy on Spark with Jupyter - Zheng preprocessing recipe

This notebook runs Scanpy's `recipe_zheng17` function using PySpark on the 1.3M neurons [dataset](https://support.10xgenomics.com/single-cell-gene-expression/datasets/1M_neurons) from 10x Genomics. The data is stored in Zarr format on GCS.

We need a few supporting files to run, so we clone a GitHub repository, and add it to the Python path.

In [1]:
! git clone https://github.com/lasersonlab/single-cell-experiments
import sys
sys.path.append("single-cell-experiments")

fatal: destination path 'single-cell-experiments' already exists and is not an empty directory.


We also need to make the files available to the executors. Note that `sc` is the Spark context.

In [2]:
sc.addPyFile("single-cell-experiments/anndata_spark.py")
sc.addPyFile("single-cell-experiments/scanpy_spark.py")
sc.addPyFile("single-cell-experiments/zarr_spark.py")

Import `AnnDataRdd` which allows us to read and write AnnData objects in Zarr format on GCS.
And `scanpy_spark` which is a re-working of some of Scanpy's preprocessing module to work on Spark.

In [3]:
from anndata_spark import AnnDataRdd
from scanpy_spark import *

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Load the AnnData object from GCS, run the Zheng recipe, then write the results back out to GCS in Zarr format.

In [4]:
%%time
adata_rdd = AnnDataRdd.from_zarr_gcs(sc, 'll-sc-data-bkup/10x/anndata_zarr/10x.zarr', 'hca-scale')
recipe_zheng17(adata_rdd)
adata_rdd.write_zarr_gcs('ll-sc-data-bkup/10x/anndata_zarr/10x-log1p.zarr', (10000, adata_rdd.adata.n_vars), 'hca-scale')

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
running recipe zheng17
the 1000 top genes correspond to a normalized dispersion cutoff of
    finished
    finished


/opt/conda/lib/python3.6/site-packages/zarr/storage.py:376: UserWarning: an object_codec is only needed for object arrays
  warnings.warn('an object_codec is only needed for object arrays')
/opt/conda/lib/python3.6/site-packages/zarr/storage.py:376: UserWarning: an object_codec is only needed for object arrays
  warnings.warn('an object_codec is only needed for object arrays')


CPU times: user 2.8 s, sys: 308 ms, total: 3.11 s
Wall time: 12min 23s
